In [1]:
import requests
from bs4 import BeautifulSoup
import re
import lxml.html as lh
import pandas as pd

In [2]:
test_url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=5000&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&showvgchartzscore=1&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showcriticscore=1&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showuserscore=1&showjapansales=0&showjapansales=1&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1'
url = test_url
response = requests.get(url)

#response.status_code
page = response.text
soup = BeautifulSoup(page, "lxml")
doc = lh.fromstring(response.content)
elements = doc.xpath('//*[@id="generalBody"]/table')[0]


In [3]:
[len(T) for T in elements[:15]]
##looks like table begins at index 2. 


[2, 1, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]

In [4]:
names = list()
for row in elements.xpath('.//tr'):
    for td in row.xpath('.//td'):
        if not td.find('a') is None:
            names.append(td.find('a').text.strip()) 
row_dict={}
df=pd.DataFrame()
row_list= list()
for counter,row in enumerate(elements.xpath(".//tr")):
    if counter > 2:
        row_list=[td.text for td in row.xpath(".//td")]
        row_dict[counter] = row_list

df=pd.DataFrame.from_dict(row_dict).transpose()
df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
             'critic_score','user_score','total_shipped','total_sales',\
              'na_sales','pal_sales','japan_sales','other_sales',\
              'release_date','last_update']

consoles = list()
for img in soup.find_all('img'):
    if 'images/consoles'in img['src']:
        console_tag = (img['src'][17:-6])
        consoles.append(img['alt'])
        
df=df.reset_index().drop(columns = ['index','blank'])
df['console'] = consoles
df['game'] = names

In [5]:
df

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
0,1,Tetris,Series,The Tetris Company,Alexey Pajitnov,N/A,N/A,N/A,496.40m,N/A,N/A,N/A,N/A,N/A,01st Jan 88,27th Feb 20
1,2,Pokemon,Series,Nintendo,Game Freak,N/A,N/A,N/A,464.84m,N/A,N/A,N/A,N/A,N/A,28th Sep 98,03rd Feb 20
2,3,Call of Duty,Series,Activision,Infinity Ward,N/A,N/A,N/A,400.00m,N/A,N/A,N/A,N/A,N/A,29th Oct 03,03rd Feb 20
3,4,Super Mario,Series,Nintendo,Nintendo,N/A,N/A,N/A,398.51m,N/A,N/A,N/A,N/A,N/A,20th Jul 83,20th Feb 20
4,5,Grand Theft Auto,Series,Rockstar Games,Rockstar North,N/A,N/A,N/A,395.00m,N/A,N/A,N/A,N/A,N/A,27th Mar 98,03rd Feb 20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,Final Fantasy VII International,PS,Square,SquareSoft,N/A,N/A,N/A,N/A,0.63m,N/A,N/A,0.59m,0.04m,02nd Oct 97,N/A
4996,4997,Cooking Mama 4: Kitchen Magic!,3DS,Majesco,Cooking Mama Ltd.,5.3,N/A,N/A,N/A,0.63m,0.50m,0.07m,N/A,0.06m,15th Nov 11,N/A
4997,4998,NHL 2001,PS,EA Sports,Page 44 Studios,N/A,N/A,N/A,N/A,0.63m,0.35m,0.24m,N/A,0.04m,26th Sep 00,N/A
4998,4999,Transformers: Revenge of the Fallen,X360,Activision,Luxoflux Corp.,7.0,6.1,N/A,N/A,0.63m,0.34m,0.23m,N/A,0.06m,23rd Jun 09,N/A


In [6]:
# Write a function that takes in a VGChartz URL and gives us all the data in their video game database
def scrape_vgchartz_videogame_db_page(url):
    
    response = requests.get(url)

    ### Check the Status
    assert(response.status_code == 200)," Website not OK " # status code = 200 => OK
    
    #Store the contents of the website under doc
    page=response.text
    soup = BeautifulSoup(page, "lxml")
    doc = lh.fromstring(response.content)
    
    # Selects the table with all the data in it on HTML using xpath
    target_table_path = doc.xpath('//*[@id="generalBody"]/table')[0]

    # Find column values that won't be scraped correctly with .text option
    names_list = find_names_column(target_table_path)
    consoles = find_console_tags(soup)
    
    # Parse non-image and non-URL info from the data table to a pandas DataFrame
    row_dict={}
    df=pd.DataFrame()
    row_list= list()
    for counter,row in enumerate(target_table_path.xpath(".//tr")):
        if counter > 2: # To skip header rows
            row_list=[td.text for td in row.xpath(".//td")]
            row_dict[counter] = row_list

    df=pd.DataFrame.from_dict(row_dict).transpose()
    df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
                 'critic_score','user_score','total_shipped','total_sales',\
                  'na_sales','pal_sales','japan_sales','other_sales',\
                  'release_date','last_update']
    
    # Correct the console and game columns using scraped values
    
    df=df.reset_index().drop(columns = ['index','blank'])
    df['console'] = consoles
    df['game'] = names_list
    return df
# Find the names of games from the links
def find_names_column(table_path):
    names_list = list()
    for row in table_path.xpath('.//tr'):
        for td in row.xpath('.//td'):
            if not td.find('a') is None:
                names_list.append(td.find('a').text.strip()) 
    return names_list
def find_console_tags(soup):
    # Console tags are stored as images, so we find the image tag and record its 'alt' value as text
    consoles = list()
    for img in soup.find_all('img'):
        if 'images/consoles'in img['src']:
            # Cut file path elements from string
            console_tag = (img['src'][17:-6])
            consoles.append(img['alt'])
    return consoles
# We can 'hack' the URL to display any number of results per page. I'll leave it as an argument.
def scrape_all_vg_chartz_videogame_db(results_per_page):
    df = pd.DataFrame()
    current_page = 1
    games_left = True
    while games_left:
        url = 'http://www.vgchartz.com/games/games.php?page=' + str(current_page) +\
        '&results=' + str(results_per_page) + '&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership\
        =Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction\
        =DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&\
        showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&\
        showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'
        new_df = scrape_vgchartz_videogame_db_page(url)
        df = df.append(new_df)

        #REMOVE LATER, TEST CONDIITON
      #  if current_page > 3:
       #     games_left = False
        print('Scraped page: ',current_page)
        if new_df.shape[0] < results_per_page:
            games_left = False
        current_page +=1
    print('Scraping done!')
    print('Total rows parsed = ', df.shape[0])
    return df.reset_index().drop(columns = 'index')


In [7]:
df=scrape_all_vg_chartz_videogame_db(10000)

<ipython-input-6-59c40f78f7e0>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Scraped page:  1


<ipython-input-6-59c40f78f7e0>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Scraped page:  2


<ipython-input-6-59c40f78f7e0>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


...

<ipython-input-6-59c40f78f7e0>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Scraped page:  5


<ipython-input-6-59c40f78f7e0>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


Scraped page:  6
Scraped page:  7
Scraping done!
Total rows parsed =  62694


<ipython-input-6-59c40f78f7e0>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_df)


In [8]:
df

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
0,1,Tetris,Series,The Tetris Company,Alexey Pajitnov,N/A,N/A,N/A,496.40m,N/A,N/A,N/A,N/A,N/A,01st Jan 88,27th Feb 20
1,2,Pokemon,Series,Nintendo,Game Freak,N/A,N/A,N/A,464.84m,N/A,N/A,N/A,N/A,N/A,28th Sep 98,03rd Feb 20
2,3,Call of Duty,Series,Activision,Infinity Ward,N/A,N/A,N/A,400.00m,N/A,N/A,N/A,N/A,N/A,29th Oct 03,03rd Feb 20
3,4,Super Mario,Series,Nintendo,Nintendo,N/A,N/A,N/A,398.51m,N/A,N/A,N/A,N/A,N/A,20th Jul 83,20th Feb 20
4,5,Grand Theft Auto,Series,Rockstar Games,Rockstar North,N/A,N/A,N/A,395.00m,N/A,N/A,N/A,N/A,N/A,27th Mar 98,03rd Feb 20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62689,62690,Zombieland: Double Tap - Road Trip,PC,GameMill Entertainment,High Voltage Software,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,15th Oct 19,25th Sep 20
62690,62691,Zombillie,NS,Forever Entertainment S.A.,Forever Entertainment S.A.,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,29th Mar 18,26th Mar 18
62691,62692,Zone of the Enders: The 2nd Runner MARS,PC,Konami,Cygames,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,06th Sep 18,26th Sep 18
62692,62693,Zoo Tycoon: Ultimate Animal Collection,XOne,Microsoft Studios,Frontier Developments,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,31st Oct 17,26th Sep 18


In [15]:
# df = df.drop(["vgchart_score"], axis = 1)
# df = df.drop(["critic_score"], axis = 1)
# df = df.drop(["user_score"], axis = 1)
# df = df.drop(['last_update'], axis = 1)
df_c = df.copy()
df_c.columns
#df.columns

Index(['position', 'game', 'console', 'publisher', 'developer',
       'total_shipped', 'total_sales', 'na_sales', 'pal_sales', 'japan_sales',
       'other_sales', 'release_date'],
      dtype='object')

In [18]:
import numpy as np
# df = df.replace("NaN", np.nan)
# df = df.dropna(subset = ['total_shipped'], how = 'any')
# df = df.dropna(thresh = 1)
df_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   position       0 non-null      object 
 1   game           0 non-null      object 
 2   console        0 non-null      object 
 3   publisher      0 non-null      object 
 4   developer      0 non-null      object 
 5   total_shipped  0 non-null      float64
 6   total_sales    0 non-null      object 
 7   na_sales       0 non-null      object 
 8   pal_sales      0 non-null      object 
 9   japan_sales    0 non-null      object 
 10  other_sales    0 non-null      object 
 11  release_date   0 non-null      object 
dtypes: float64(1), object(11)
memory usage: 0.0+ bytes


In [17]:
import pandas as pd

# Replace 'M' and 'K' suffixes with appropriate multipliers
df_c['total_shipped'] = df_c['total_shipped'].str.replace('M', 'e6').str.replace('K', 'e3')

# Convert total_shipped to numeric type
df_c['total_shipped'] = pd.to_numeric(df_c['total_shipped'], errors='coerce')

# Drop rows with NaN values in total_shipped column
df_c = df_c.dropna(subset=['total_shipped'])
df_c

,position,game,console,publisher,developer,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date


In [19]:
import pandas as pd

# Convert total_shipped to numeric type
df_c['total_shipped'] = pd.to_numeric(df_c['total_shipped'], errors='coerce')

# Drop rows with NaN values in total_shipped column
df_c = df_c.dropna(subset=['total_shipped'])

# Print number of rows and non-null values in total_shipped column
print(f"Number of rows in dataframe: {len(df_c)}")
print(f"Number of non-null values in total_shipped column: {df_c['total_shipped'].count()}")


Number of rows in dataframe: 0
Number of non-null values in total_shipped column: 0


In [21]:
df1 = df.copy()

In [24]:
def numeric_converter(x):
    suffixes = {'m': 1e6, 'k': 1e3, 'M': 1e6, 'K': 1e3}
    if type(x) == str:
        if x.lower() == 'n/a':
            return np.nan
        elif x[-1] in suffixes.keys():
            return float(x[:-1]) * suffixes[x[-1]]
        else:
            return float(x)
    else:
        return x
# Convert total_shipped to numeric type with custom converter
df1['total_shipped'] = pd.to_numeric(df1['total_shipped'].apply(numeric_converter), errors='coerce')

# Drop rows with NaN values in total_shipped column
df1 = df1.dropna(subset=['total_shipped'])
df1


,position,game,console,publisher,developer,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date
0,1,Tetris,Series,The Tetris Company,Alexey Pajitnov,496400000.0,N/A,N/A,N/A,N/A,N/A,01st Jan 88
1,2,Pokemon,Series,Nintendo,Game Freak,464840000.0,N/A,N/A,N/A,N/A,N/A,28th Sep 98
2,3,Call of Duty,Series,Activision,Infinity Ward,400000000.0,N/A,N/A,N/A,N/A,N/A,29th Oct 03
3,4,Super Mario,Series,Nintendo,Nintendo,398510000.0,N/A,N/A,N/A,N/A,N/A,20th Jul 83
4,5,Grand Theft Auto,Series,Rockstar Games,Rockstar North,395000000.0,N/A,N/A,N/A,N/A,N/A,27th Mar 98
...,...,...,...,...,...,...,...,...,...,...,...,...
22504,22505,FireTeam,PC,Unknown,Multitude,0.0,N/A,N/A,N/A,N/A,N/A,N/A
22701,22702,Hi-Res Computer Golf,OSX,Unknown,Avant-Garde Creations,0.0,N/A,N/A,N/A,N/A,N/A,N/A
22704,22705,Ski Crazed,ApII,Baudville,Naughty Dog,0.0,N/A,N/A,N/A,N/A,N/A,01st Oct 87
22706,22707,Super Stellar Trek,OSX,Unknown,"Rainbow Computing, Inc.",0.0,N/A,N/A,N/A,N/A,N/A,N/A


In [25]:
# Remove rows where total_shipped is 0.0
df1 = df1[df1['total_shipped'] != 0.0]
df1


,position,game,console,publisher,developer,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date
0,1,Tetris,Series,The Tetris Company,Alexey Pajitnov,496400000.0,N/A,N/A,N/A,N/A,N/A,01st Jan 88
1,2,Pokemon,Series,Nintendo,Game Freak,464840000.0,N/A,N/A,N/A,N/A,N/A,28th Sep 98
2,3,Call of Duty,Series,Activision,Infinity Ward,400000000.0,N/A,N/A,N/A,N/A,N/A,29th Oct 03
3,4,Super Mario,Series,Nintendo,Nintendo,398510000.0,N/A,N/A,N/A,N/A,N/A,20th Jul 83
4,5,Grand Theft Auto,Series,Rockstar Games,Rockstar North,395000000.0,N/A,N/A,N/A,N/A,N/A,27th Mar 98
...,...,...,...,...,...,...,...,...,...,...,...,...
21831,21832,Horizon V,All,Gebelli Software,Gebelli Software,10000.0,N/A,N/A,N/A,N/A,N/A,01st Jan 82
21832,21833,Asylum,PC,Unknown,William F. Denman Jr.,10000.0,N/A,N/A,N/A,N/A,N/A,N/A
21833,21834,DragonQuest,All,Enix,Simulations Publications,10000.0,N/A,N/A,N/A,N/A,N/A,27th May 86
21835,21836,Pac Attack,PC,Unknown,Computerware,10000.0,N/A,N/A,N/A,N/A,N/A,N/A


In [26]:
# Use value_counts() method to count unique values in the column
sales_counts = df1['total_sales'].value_counts()

# Print the counts of each unique value
sales_counts

N/A    4212
Name: total_sales, dtype: int64

In [27]:
#Since all total_sales values are N/A - I will drop the total_sales, na_sales, pal_sales, japan_sales and other_sales. 
df2 = df1.drop(['total_sales', 'na_sales','pal_sales', 'japan_sales', 'other_sales'], axis = 1)
df2

,position,game,console,publisher,developer,total_shipped,release_date
0,1,Tetris,Series,The Tetris Company,Alexey Pajitnov,496400000.0,01st Jan 88
1,2,Pokemon,Series,Nintendo,Game Freak,464840000.0,28th Sep 98
2,3,Call of Duty,Series,Activision,Infinity Ward,400000000.0,29th Oct 03
3,4,Super Mario,Series,Nintendo,Nintendo,398510000.0,20th Jul 83
4,5,Grand Theft Auto,Series,Rockstar Games,Rockstar North,395000000.0,27th Mar 98
...,...,...,...,...,...,...,...
21831,21832,Horizon V,All,Gebelli Software,Gebelli Software,10000.0,01st Jan 82
21832,21833,Asylum,PC,Unknown,William F. Denman Jr.,10000.0,N/A
21833,21834,DragonQuest,All,Enix,Simulations Publications,10000.0,27th May 86
21835,21836,Pac Attack,PC,Unknown,Computerware,10000.0,N/A


In [29]:
# Save the updated DataFrame as a new CSV file
df2.to_csv('game_shipped.csv', index=False)
